In [1]:
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}
import geotrellis.spark.io.hadoop._
import org.apache.spark.rdd.RDD
import org.apache.hadoop.io._
import org.apache.hadoop.io.{IOUtils, SequenceFile}
import sys.process._


In [2]:
print("test")



test

## Configuration

In [3]:

var _dir_path_hdfs = "hdfs:///user/hadoop/mycluster0/"
var file_name = "listfile.txt"
var file_name_dir :String = _dir_path_hdfs + file_name

var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

if (fs.exists(new org.apache.hadoop.fs.Path(file_name_dir))) {
    println("The file " + file_name_dir + " is found! :))")
}else{
    println("Put the file in user/hadoop/mycluster0 on the HDFS")
}

// Connect to Spark
var appName = "phenolohy_timesat"
var masterURL = "spark://mycluster0.mydomain:7077"

//A context needs to be created if it does not already exist
val conf_spark = new SparkConf().setAppName(appName).setMaster(masterURL)
val sc_spark = new SparkContext(conf_spark)

// Test
val info = List(("viktor", 24), ("joe", 30), ("jack", 30))
val infoRDD = sc_spark.parallelize(info)
infoRDD.collect().foreach(println)





Waiting for a Spark session to start...

The file hdfs:///user/hadoop/mycluster0/listfile.txt is found! :))
(viktor,24)                                                                     
(joe,30)
(jack,30)


_dir_path_hdfs = hdfs:///user/hadoop/mycluster0/
file_name = listfile.txt
file_name_dir = hdfs:///user/hadoop/mycluster0/listfile.txt
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_207783209_39, ugi=mycluster (auth:SIMPLE)]]
appName = phenolohy_timesat
masterURL = spark://mycluster0.mydomain:7077
conf_spark = org.apache.spark.SparkConf@51de6bf8
sc_spark = org.apache.spark.SparkContext@2ba34c24


in...


Spark app-20180315231501-0012: Some(http://192.168.0.101:4041)

In [64]:
def start_process_single_timmesat(): Unit = {
  println("barrrrrrrrrr")
}

start_process_single_timmesat()


start_process_single_timmesat: ()Unit


barrrrrrrrrr
